<a href="https://colab.research.google.com/github/shah-zeb-naveed/autogen-udemy-course/blob/main/Excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyautogen -q

# Agents

In [ ]:
from google.colab import userdata
from autogen import ConversableAgent

In [ ]:
llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature" : 0.7
}

In [ ]:
audience = ConversableAgent(
    name = "audience",
    llm_config = llm_config,
    is_termination_msg = lambda msg: 'HAHA' in msg['content'].upper(),
    system_message = 'You are a member of the audience of a comedy show that is hard to impress.'
)

In [ ]:
comedian = ConversableAgent(
    name = "comedian",
    llm_config = llm_config,
    max_consecutive_auto_reply=2,
    system_message = 'You are a comedian that tells bad jokes.'
)

In [ ]:
comedian.initiate_chat(
    audience,
    message="Welcome to my standup comedy show! Are you ready for a night full of laughter?",
    max_turns = 2
)

comedian (to audience):

Welcome to my standup comedy show! Are you ready for a night full of laughter?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
audience (to comedian):

We'll see about that. Impress us with your jokes!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
comedian (to audience):

Why did the tomato turn red? Because it saw the salad dressing!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
audience (to comedian):

*polite chuckle* Not bad, not bad. Keep 'em coming.

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Welcome to my standup comedy show! Are you ready for a night full of laughter?', 'role': 'assistant'}, {'content': "We'll see about that. Impress us with your jokes!", 'role': 'user'}, {'content': 'Why did the tomato turn red? Because it saw the salad dressing!', 'role': 'assistant'}, {'content': "*polite chuckle* Not bad, not bad. Keep 'em coming.", 'role': 'user'}], summary="*polite chuckle* Not bad, not bad. Keep 'em coming.", cost={'usage_including_cached_inference': {'total_cost': 0.0006180000000000001, 'gpt-3.5-turbo-0125': {'cost': 0.0006180000000000001, 'prompt_tokens': 720, 'completion_tokens': 172, 'total_tokens': 892}}, 'usage_excluding_cached_inference': {'total_cost': 0.00015450000000000001, 'gpt-3.5-turbo-0125': {'cost': 0.00015450000000000001, 'prompt_tokens': 180, 'completion_tokens': 43, 'total_tokens': 223}}}, human_input=[])

# Human-in-the-loop

In [ ]:
from google.colab import userdata
from autogen import ConversableAgent

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

In [ ]:
agent_guesser = ConversableAgent(
    name="agent_guesser",
    system_message="""Let's play a game. I have a person in mind, and you have to guess it.
    I'll respond with 'yes' or 'no' with a hint. Don't ask questions just make best
    use of information you already have received and return person's name. Go ahead and start guessing!""",
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies to a particular recipient
    llm_config=llm_config,
    human_input_mode="TERMINATE", # ask for human input at termination
)

agent_thinker = ConversableAgent(
    "agent_thinker",
    system_message="""Let's play a game. Think of a famous personality, and I'll try to guess their name on every turn.
    Respond with 'yes' or 'no'. If no, also provide a hint. Let's get started!""",
    llm_config=llm_config,
    human_input_mode="NEVER",  # default
)


In [ ]:
result = agent_thinker.initiate_chat(
    agent_guesser,
    message="Let's start. I have someone in mind."
)

agent_thinker (to agent_guesser):

Let's start. I have someone in mind.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_guesser (to agent_thinker):

Is the person you're thinking of a fictional character?

--------------------------------------------------------------------------------
agent_thinker (to agent_guesser):

No.

--------------------------------------------------------------------------------
Please give feedback to agent_thinker. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation: is this a politician?
agent_guesser (to agent_thinker):

is this a politician?

--------------------------------------------------------------------------------
agent_thinker (to agent_guesser):

Yes.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_guesser (to agent_thinker):

Is the person you're thinking of a current world leade

# Code Executors

In [ ]:
from google.colab import userdata

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor


executor = LocalCommandLineCodeExecutor(
    timeout=10,
    work_dir='code/'
)

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
)

message_with_code_block = """
Here's a code that solves your problem:
```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create a DataFrame with sample data
data = pd.DataFrame({
    'x': np.arange(10),
    'y': np.random.randint(0, 10, 10)
})

# Plot the line plot
plt.plot(data['x'], data['y'])
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Line Plot')
plt.savefig('lineplot.png')
plt.show()
print('Line plot saved to lineplot.png')
```
"""

reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Figure(640x480)
Line plot saved to lineplot.png



In [ ]:
code_assistant_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

code_assistant = ConversableAgent(
    "code_assistant",
    llm_config=llm_config,
    system_message=code_assistant_system_message,
    code_execution_config=False,  # Turn off code execution for this agent.
)

In [ ]:
chat_result = code_executor_agent.initiate_chat(
    code_assistant,
    message="Write Python code to calculate the 23rd prime number",
    max_turns=2
)

code_executor_agent (to code_assistant):

Write Python code to calculate the 23rd prime number

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_assistant (to code_executor_agent):

```python
# Calculate the 23rd prime number
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

count = 0
num = 2

while count < 23:
    if is_prime(num):
        count += 1
    num += 1

print(num - 1)
```


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to code_assistant):

exitcode: 0 (execution succeeded)
Code output: 83


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_assistant (to code_executor_agent):

The 23rd prime number is 83. The code

# Tools

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_wiki_main_page(url: str) -> str:
    """
    Scrapes news content from wikipedia's main page.

    Args:
        url (str): The URL of the wikipedia page to scrape.

    Returns:
        str: The text content of the webpage.
             Returns None if there is an error during the process.
    """

    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            content = soup.get_text()
            return content
            content = content[content.find("In the news"):content.find("Ongoing")]
            return content

        else:
            print("Failed to retrieve webpage. Status code:", response.status_code)
            return None
    except Exception as e:
        print("An error occurred:", e)
        return None


print(scrape_wiki_main_page('https://en.wikipedia.org/wiki/Main_Page'))





Wikipedia, the free encyclopedia






































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload file



















Search











Search





























Create account

Log in








Personal tools





 Create account Log in





		Pages for logged out editors learn more



ContributionsTalk



























Main Page










Main PageTalk





English

















ReadView sourceView history







Tools





Tools
move to sidebar
hide



		Actions
	


ReadView sourceView history





		General
	


What links hereRelated changesUpload fileSpecial pagesPermanent linkPage informationCite this pageGet shortened URLDownload QR codeWikidata item





		Print/export
	


Download as PDFPrintable version





		In other projects
	


Wiki

In [ ]:
import os
from google.colab import userdata
from autogen import ConversableAgent

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="assistant",
    llm_config=llm_config,
    system_message="""You are a helpful AI news bot.
    You can pull content from wikipedia's main page.
    URL: https://en.wikipedia.org/wiki/Main_Page. Once pulled, only share the top 3 stories.
    Return 'TERMINATE' when the task is done.""",
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="user_proxy",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="NEVER",
)

# Register the tool's information with the assistant agent.
assistant.register_for_llm(
    name="scrape_wiki_main_page",
    description="A tool for scraping today's news from wikipedia"
)(scrape_wiki_main_page)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(
    name="scrape_wiki_main_page"
)(scrape_wiki_main_page)

In [ ]:
from autogen import Cache

with Cache.disk() as cache:
  chat_result = user_proxy.initiate_chat(
      assistant,
      message="What's hot in today's news?",
      max_turns=5
  )

user_proxy (to assistant):

What's hot in today's news?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to user_proxy):

***** Suggested tool call (call_QA4w3HFGiCTYHv6AJjTnNgkR): scrape_wiki_main_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Main_Page"}
**************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_wiki_main_page...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_QA4w3HFGiCTYHv6AJjTnNgkR) *****
In the news


Manasseh Sogavare

The Ownership, Unity and Responsibility Party, led by Prime Minister Manasseh Sogavare (pictured), wins the most seats in the Solomon Islands general election but falls short of a majority.
NASA announces that the Voyager 1 space probe is sending readable data for the fir

In [ ]:
from autogen import Cache

with Cache.disk() as cache:
  chat_result = user_proxy.initiate_chat(
      assistant,
      message="What's hot in today's news?",
      max_turns=2
  )

print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

chat_result.summary

# Sequential Chats

In [ ]:
from autogen import ConversableAgent
from google.colab import userdata

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

In [ ]:
# A quest where an adventurer seeks help from wizard
# to get past the guardian of the secret vault

# Define the agents

# The Adventurer Agent seeks advice from a Wizard on magical matters.
adventurer_agent = ConversableAgent(
    name="Adventurer_Agent",
    system_message="""
    You are a worthy adventurous hero seeking advice from a Wizard on magical matters.
    Briefly communicate in less than 10 words.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The Wizard Agent is an expert in magic and provides magical advice to the Adventurer.
wizard_agent = ConversableAgent(
    name="Wizard_Agent",
    system_message="""
    You are a wise and powerful Wizard. You provide magical advice to the
    Adventurer. You know the passcode to the vault is Enigma.
    Briefly communicate in less than 10 words.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# The guardian Agent guards the hidden gem and offers protection to the Adventurer.
guardian_agent = ConversableAgent(
    name="Guardian_Agent",
    system_message="""You are the Guardian of the Hidden Gem, sworn to protect it from unworthy adventurers.
    The passcode to the vault is Enigma. If adventurer mentions Enigma passcode, say 'Welcome!' otherwise say
    'Incorrect Passcode'.  Briefly communicate in less than 10 words.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [ ]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat() method.

sequence_of_chats = [
    {
      "recipient": wizard_agent,
      "message": "Help me get past the Guardian of the Hidden Gem in the secret vault.",
      "max_turns": 1,
      "summary_method": "reflection_with_llm",
    },
    {
      "recipient": guardian_agent,
      "message": "Allow me to enter the secret vault!",
      "max_turns": 1
    }
]

chat_results = adventurer_agent.initiate_chats(sequence_of_chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Whisper "Enigma" to unlock the hidden gem's magic.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Guardian_Agent):

Allow me to enter the secret vault!
Context: 
Whisper "Enigma" to unlock the hidden gem's magic.

--------------------------------------------------------------------------------
Guardian_Agent (to Adventurer_Agent):

Welcome!


# Nested Chat

In [ ]:
poetry_agent = ConversableAgent(
    name="Poet",
    system_message="You are an AI poet. Create only one stanza.",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

# parameters of initiate_chat() method
sequence_of_chats = [
    {
        "recipient": wizard_agent,
        "max_turns": 1,
        "message": "Help me get past the Guardian of the Hidden Gem in the secret vault.",
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Concisely summarize the instructions given by wizard agent to communicate with guardian.",
    },
    {
        "recipient": guardian_agent,
        "max_turns": 1,
        "message": "Allow me to enter the secret vault!",
        "summary_method": "reflection_with_llm",
        "summary_prompt": "Concisely summarize the quest.",
    },
    {
        "recipient": poetry_agent,
        "max_turns": 1,
        "message": "Write a poem on my expedition.",
        "summary_method": "last_msg",
    },
]

adventurer_agent.register_nested_chats(
    sequence_of_chats,
    # trigger determines if the agent should start nested chat given the sender agent.
    # In this case, the adventurer agent will not start the nested chats if the sender is
    # from the nested chats' recipient to avoid recursive calls.
    trigger=lambda sender: sender not in [wizard_agent, guardian_agent, poetry_agent],
)

In [ ]:
adventurer_agent.generate_reply(
    messages=[{"role": "user", "content": "Hello!"}]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Whisper "Enigma" to unlock the hidden gem's magic.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Guardian_Agent):

Allow me to enter the secret vault!
Context: 
Whisper "Enigma" to unlock the hidden gem's magic.

--------------------------------------------------------------------------------
Guardian_Agent (to Adventurer_Agent):

Welcome!


'In the heart of mystery\'s embrace, I tread\nWhispering "Enigma" to awaken the magic hidden ahead'

In [ ]:
chat_results = user_proxy.initiate_chat(adventurer_agent, max_turns=1)
chat_results


# user_proxy ignores the message and relies on input
# input routed as is to the first recipient

## Impact of Role

In [ ]:
agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [llm_config]},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    human_input_mode="NEVER",  # Never ask for human input.
)

reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)

reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "assistant"}])
print(reply)

# role doesn't make a difference

# Group Chat

In [ ]:
from autogen import ConversableAgent
from google.colab import userdata

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

In [ ]:
# Define the roles and their descriptions
# a software engineering team:
# scrum master
# software engineer
# product manager

product_manager = ConversableAgent(
    name="product_manager",
    system_message="You are an expert Product Manager working in a scrum team of an IT company. You like innovation and introducing new features as soon as possible.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    description="Provides product requirements and priorities."
)

scrum_master = ConversableAgent(
    name="scrum_master",
    system_message="You are a scrum master leading the 14-day sprints in an IT company. You provide technical leadership and make sure work is apporpirately planned.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    description="Leads the sprint planning process."
)

software_engineer = ConversableAgent(
    name="software_engineer",
    system_message="You are a senior software engineer in an IT company. You write software and technical documentation.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    description="Provides insights on technical feasibility and effort estimation."
)


In [ ]:
allowed_transitions = {
    product_manager: [scrum_master],
    scrum_master: [software_engineer, product_manager],
    software_engineer: [scrum_master],
}

In [ ]:
from autogen import GroupChat, GroupChatManager

# Create a GroupChat object and provide the list of agents
sprint_planning_chat = GroupChat(
    agents=[product_manager, scrum_master, software_engineer],
    messages=[],
    max_round=5,  # Setting a maximum round for the conversation,
    send_introductions=True,
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
)

# Create a GroupChatManager object and provide the GroupChat object as input
sprint_planning_chat_manager = GroupChatManager(
    groupchat=sprint_planning_chat,
    llm_config=llm_config  # Assuming the use of GPT-4 model
)


# Initiate the chat with the product_manager as the starting speaker
chat_result = product_manager.initiate_chat(
    sprint_planning_chat_manager,
    message="We have a request to develop a website to track employee HR requests. How do we go about it?"
)


product_manager (to chat_manager):

We have a request to develop a website to track employee HR requests. How do we go about it?

--------------------------------------------------------------------------------
scrum_master (to chat_manager):

Hello! To develop a website to track employee HR requests, we can start by breaking down the requirements into user stories or tasks. We will work closely with the product manager to understand the requirements and priorities. Next, the software engineers can provide insights on the technical feasibility and effort estimation for each task. Based on this information, we can plan out the tasks for the upcoming sprint and allocate resources accordingly. Does that sound good to everyone? Let's collaborate and get started on this project.

--------------------------------------------------------------------------------
software_engineer (to chat_manager):

As a software engineer, I can offer my technical expertise to ensure that the website to track 

## Custom Speaker Selection

In [ ]:
def custom_speaker_selection(last_speaker, groupchat):
    messages = groupchat.messages

    # decide next turn
    if last_speaker is scrum_master:
        return product_manager
    elif last_speaker is software_engineer:
        return scrum_master
    elif last_speaker is product_manager:
        return software_engineer

    return None


sprint_planning_chat = GroupChat(
    agents=[product_manager, scrum_master, software_engineer],
    messages=[],
    max_round=3,
    send_introductions=True,
    speaker_selection_method=custom_speaker_selection,
)

sprint_planning_chat_manager = GroupChatManager(
    groupchat=sprint_planning_chat,
    llm_config=llm_config
)

# Initiate the chat
chat_result = product_manager.initiate_chat(
    sprint_planning_chat_manager,
    message="We have a request to develop a system to track employee HR requests. How do we go about it?",
    summary_method="reflection_with_llm", # what's the point of this?
)


product_manager (to chat_manager):

We have a request to develop a system to track employee HR requests. How do we go about it?

--------------------------------------------------------------------------------
software_engineer (to chat_manager):

Hello, product_manager. To develop a system to track employee HR requests, we will need to gather detailed requirements from you. This will include understanding the types of HR requests to be tracked, the data to be captured for each request, user roles and permissions, workflow processes, reporting requirements, and any integrations with existing systems. Once we have a clear understanding of the requirements, we can work on designing the system architecture, identifying the technologies to be used, and estimating the development effort. It would be helpful if you could provide us with a detailed overview of the HR request tracking system you envision so that we can proceed with planning and implementation.

--------------------------------

# Text Transformation

In [ ]:
import copy
import pprint

from google.colab import userdata
import autogen
from autogen import ConversableAgent
from autogen.agentchat.contrib.capabilities import transforms, transform_messages

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

In [ ]:
max_msg_transfrom = transforms.MessageHistoryLimiter(max_messages=3)

# messages in OpenAI format
messages = [
    {"role": "user", "content": "Hello!"},
    {"role": "assistant", "content": [{"type": "text", "text": "Hi! How can I help you?"}]},
    {"role": "user", "content": "I just wanna talk..."},
    {"role": "assistant", "content": [{"type": "text", "text": "Sure, I'm here for you. Sure, I'm here for you. Sure, I'm here for you. Sure, I'm here for you. "}]},
    {"role": "user", "content": "Idk..."},
]

processed_messages = max_msg_transfrom.apply_transform(copy.deepcopy(messages))
pprint.pprint(processed_messages)

[{'content': 'I just wanna talk...', 'role': 'user'},
 {'content': [{'text': "Sure, I'm here for you. Sure, I'm here for you. Sure, "
                       "I'm here for you. Sure, I'm here for you. ",
               'type': 'text'}],
  'role': 'assistant'},
 {'content': 'Idk...', 'role': 'user'}]


In [ ]:
token_limit_transform = transforms.MessageTokenLimiter(max_tokens_per_message=3)
processed_messages = token_limit_transform.apply_transform(copy.deepcopy(messages))
pprint.pprint(processed_messages)

[{'content': 'Hello!', 'role': 'user'},
 {'content': [{'text': 'Hi! How', 'type': 'text'}], 'role': 'assistant'},
 {'content': 'I just wanna', 'role': 'user'},
 {'content': [{'text': 'Sure, I', 'type': 'text'}], 'role': 'assistant'},
 {'content': 'Idk...', 'role': 'user'}]


In [ ]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config=llm_config,
)

transformer_handler = transform_messages.TransformMessages(
    transforms=[
        transforms.MessageHistoryLimiter(max_messages=5),
        transforms.MessageTokenLimiter(max_tokens=100, max_tokens_per_message=5),
    ]
)

transformer_handler.add_to_agent(assistant)

# assistant will now received transformed messages!

In [ ]:
import copy
import re
from typing import Dict, List

import autogen
from autogen.agentchat.contrib.capabilities import transform_messages, transforms

# apply_transform follows transform_messages.MessageTransform protocol
class PIIRemover:
    def __init__(self):
        self._email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
        self._replacement_string = 'REDACTED'

    def apply_transform(self, messages: List[Dict]) -> List[Dict]:
        temp_messages = copy.deepcopy(messages)

        for message in temp_messages:
            if isinstance(message["content"], str):
                message["content"] = re.sub(self._email_pattern, self._replacement_string, message["content"])
            elif isinstance(message["content"], list):
                for item in message["content"]:
                    if item["type"] == "text":
                        item["text"] = re.sub(self._email_pattern, self._replacement_string, item["text"])
        return temp_messages

    # borrowed from autogen MessageHistoryLimiter
    def get_logs(self, pre_transform_messages, post_transform_messages):
        pre_transform_messages_len = len(pre_transform_messages)
        post_transform_messages_len = len(post_transform_messages)

        if post_transform_messages_len < pre_transform_messages_len:
            logs_str = (
                f"Removed {pre_transform_messages_len - post_transform_messages_len} messages. "
                f"Number of messages reduced from {pre_transform_messages_len} to {post_transform_messages_len}."
            )
            return logs_str, True
        return "No messages were removed.", False


In [ ]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

tranformer_handler = transform_messages.TransformMessages(transforms=[PIIRemover()])
tranformer_handler.add_to_agent(assistant)

In [ ]:
message = {
    "content": "contact support team at example_email@example.com for more queries.",
    "role" : 'user'
}

assistant.generate_reply(
    messages=[message]
)

'I cannot directly contact the support team for you. However, you can reach out to them yourself by sending an email to the provided email address "REDACTED" for any queries you may have. Is there anything else I can assist you with regarding this matter?'

# Prompt Engineering: ReAct

In [ ]:
!pip install tavily-python chromadb langchain pyautogen -q

In [ ]:
# Adapted from Langchain's ReAct agent: https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/react/agent.py#L79
ReAct_prompt = """
Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
"""

In [ ]:
import os
from typing import Annotated
from google.colab import userdata

import autogen
from autogen import ConversableAgent, UserProxyAgent
from autogen.cache import Cache
from autogen.agentchat.contrib.capabilities import transforms, transform_messages
from autogen.coding import LocalCommandLineCodeExecutor

from tavily import TavilyClient

llm_config = {
    "model": "gpt-4",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7,
    "max_tokens": 1000,
}

In [ ]:
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [ ]:
def search_tool(query: Annotated[str, "The search query"]) -> Annotated[str, "The search results"]:
    return tavily.get_search_context(query=query, search_depth="advanced")

print(search_tool(query="Ronaldo's retirement date"))

"[\"{\\\"url\\\": \\\"https://www.footballtransfers.com/en/transfer-news/uk-premier-league/2022/09/cristiano-ronaldo-retire-date-retirement\\\", \\\"content\\\": \\\"When will Cristiano Ronaldo retire? Cristiano Ronaldo will continue to play until at least 2024, the Manchester United star has confirmed. Ronaldo has revealed that he plans to play for Portugal at Euro 2024, at which point he will be 39 years old.\\\"}\", \"{\\\"url\\\": \\\"https://www.eurosport.com/football/world-cup/2022/has-cristiano-ronaldo-retired-was-qatar-2022-his-last-world-cup-which-club-does-portugal-superstar-n_sto9271431/story.shtml\\\", \\\"content\\\": \\\"There is just no keeping Cristiano Ronaldo out of the news, even now his Portugal team are no longer in the 2022 World Cup and he appears to be potentially on the brink of retirement from at least ...\\\"}\", \"{\\\"url\\\": \\\"https://www.mirror.co.uk/sport/football/news/cristiano-ronaldo-man-united-future-28045857\\\", \\\"content\\\": \\\"Ronaldo has 

In [ ]:
# create agents
user_proxy = UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().upper().endswith("TERMINATE"),
    human_input_mode="NEVER",
    code_execution_config=False#{"executor": LocalCommandLineCodeExecutor(work_dir="coding")},
)

In [ ]:
assistant = autogen.AssistantAgent(
    name="Assistant",
    system_message="You are an AI Assistant. Only use tools you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

In [ ]:
# register tool
autogen.register_function(
    search_tool,
    caller=assistant,
    executor=user_proxy,
    name="search_tool",
    description="Search the web for the given query",
)

In [ ]:
def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

In [ ]:
with Cache.disk(cache_seed=5) as cache:
  user_proxy.initiate_chat(
    assistant,
    question="Why is NVIDIA's stock down today",
    max_turns=4,
    message=react_prompt_message
)

user_proxy (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: Why is NVIDIA's stock down today


--------------------------------------------------------------------------------
Assistant (to user_proxy):

Thought: I need to look up the latest news or reports for NVIDIA to understand why their stock is down today.
Action: Use the search_tool function to find information about NVIDIA's stock performance.
Action Input: { "query": "Why is NVIDIA's stock down today" }

--------------------------------------------------------------------------------
user_proxy 

In [ ]:
# def generate_tool_schema(tool):
#     """
#     Define the tool's schema based on tool.args_schema for LLM config
#     """
#     tool_schema = {
#         "name": tool.name.lower().replace(" ", "_"),
#         "description": tool.description,
#         "parameters": {
#             "type": "object",
#             "properties": {},
#             "required": [],
#         },
#     }

#     if tool.args is not None:
#         tool_schema["parameters"]["properties"] = tool.args

#     return tool_schema

# generate_tool_schema(search_tool)

# llm_config = {
#     "functions": [generate_tool_schema(search_tool)],
#     "config_list": [llm_config],
#     "timeout" : 120
# }

# ReAct with Teachability

In [ ]:
from autogen.agentchat.contrib.capabilities import teachability

# Instantiate Teachability capability
teachability = teachability.Teachability(
    verbosity=3,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    reset_db=True,
    path_to_db_dir="./teachability_db",
    recall_threshold=1.5,  # Squared L2, Higher numbers allow more (but less relevant) memos to be recalled.
)

# add Teachability capability
teachability.add_to_agent(assistant)


CLEARING MEMORY


In [ ]:
# starts with empty vector store

with Cache.disk() as cache:
  user_proxy.initiate_chat(
      assistant,
      question="What is LLAMA 3's release date?",
      max_turns=4,
      message=react_prompt_message
  )

user_proxy (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: Who is the current prime minister of germany?


--------------------------------------------------------------------------------
Assistant (to analyzer):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multip

KeyboardInterrupt: 

In [ ]:
# useful info stored in vector store now
with Cache.disk() as cache:
  user_proxy.initiate_chat(
      assistant,
      question="Who is the current prime minister of germany?",
      max_turns=4,
      message=react_prompt_message
  )

user_proxy (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: What is the result of super bowl 2024?


--------------------------------------------------------------------------------
Assistant (to user_proxy):

Question: What is the result of super bowl 2024?
Thought: Based on the information provided, the Kansas City Chiefs won Super Bowl 2024.
Final Answer: The Kansas City Chiefs won Super Bowl 2024.

--------------------------------------------------------------------------------
user_proxy (to Assistant):



-------------------------------------------

In [ ]:
# from autogen.agentchat.contrib.capabilities.teachability import MemoStore

# memo_store = MemoStore(verbosity=0, reset=False, path_to_db_dir='./teachability_db')
# memo_store.list_memos()

# Retrieval Augmented Generation (RAG)

In [ ]:
!pip install pyautogen chromadb sentence_transformers markdownify pypdf -q --progress-bar off

In [ ]:
import chromadb
from google.colab import userdata
from autogen import Cache

from autogen import ConversableAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent


llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

In [ ]:
retriever_config = {
    "task": "qa",
    "model": llm_config["model"],
    "client": chromadb.PersistentClient(path="/tmp/chromadb"),
    "embedding_model": "all-mpnet-base-v2",
    "docs_path": [
        "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/887f029a4f5b5fd30eaba844a1168141556b1c02/nasa-e-book/earth-txt-10/page-11.txt",
        "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/887f029a4f5b5fd30eaba844a1168141556b1c02/nasa-e-book/earth-txt-10/page-13.txt",
        "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/887f029a4f5b5fd30eaba844a1168141556b1c02/nasa-e-book/earth-txt-10/page-15.txt",
        "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/887f029a4f5b5fd30eaba844a1168141556b1c02/nasa-e-book/earth-txt-10/page-17.txt",
        "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/887f029a4f5b5fd30eaba844a1168141556b1c02/nasa-e-book/earth-txt-10/page-19.txt",
        "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/887f029a4f5b5fd30eaba844a1168141556b1c02/nasa-e-book/earth-txt-10/page-21.txt",
        "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/887f029a4f5b5fd30eaba844a1168141556b1c02/nasa-e-book/earth-txt-10/page-23.txt",

    ]
}

rag_proxy_agent = RetrieveUserProxyAgent(
    name="rag_proxy_agent",
    human_input_mode="NEVER",
    retrieve_config=retriever_config,
    code_execution_config=False,  # set to False if you don't want to execute any code
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
rag_assistant = RetrieveAssistantAgent(
    name="rag_assistant",
    system_message="You are a helpful AI assistant.",
    llm_config={
        "timeout": 600,
        "config_list": [llm_config],
    },
)

In [ ]:
query = "What causes the colorful ring-like phenomenon over the Pacific Ocean"

with Cache.disk() as cache:
  rag_proxy_agent.initiate_chat(
      rag_assistant,
      problem=query,
      n_results=2,
      message=rag_proxy_agent.message_generator,
      max_turns=5
  )

Trying to create collection.


2024-04-27 13:17:23,249 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 7 chunks.
2024-04-27 13:17:23,255 - autogen.agentchat.contrib.vectordb.chromadb - INFO - No content embedding is provided. Will use the VectorDB's embedding function to generate the content embedding.


VectorDB returns doc_ids:  [['f6368edd', 'e7c488a3']]
Adding content of doc f6368edd to context.
Adding content of doc e7c488a3 to context.
rag_proxy_agent (to rag_assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What causes the colorful ring-like phenomenon over the Pacific Ocean

Context is: A Glorious View	Pacific Ocean
A layer of stratocumulus clouds over the Pacific Ocean serves as the backdrop for this rainbow-like phenomenon known as a glory. 
Glories form when water droplets within clouds scatter sunlight back toward a source of illumination (in this case, the Sun). 
Although glories may look similar to rainbows, the way light is scattered to produce them is different. Rainbows are formed by refraction and r